In [1]:
from Tempo import Tempo
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)
obj = Tempo()

In [2]:
end_date = date(2018, 8, 12)
start_date = date(2018, 1, 16)

In [ ]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], single_date.strftime("%Y/%m/%d"))